In [1]:
# 02a - Création dataframe dft, qui contient les données de BDTOPO
# Charge les données de BDTOPO, les met en forme et les sauvegarde au format pickle .pkl

In [1]:
# Paths globaux
HOME = '/Users/u600141/OneDrive - La Francaise des Jeux/Data/constructions/'
PATHS_BDTOPO = HOME + 'images/BDTOPO/source/1_DONNEES_LIVRAISON_2023-06-00117/BDT_3-3_SHP_LAMB93_D092-ED2023-06-15/*/*.shp'
PATHS_BDTOPO_PICKLE = HOME + 'images/BDTOPO/pickle/'

In [2]:
# Imports
import glob
import os
import pandas as pd
import geopandas as gpd

In [5]:
# Chargement base BDTOPO : Fichiers .shp, avec un répertoire par thème (administratif, bâti, etc.)

# load_shp() : lecture d'un fichier .shp
def load_shp(path):
    # Load data
    bdtopo = gpd.read_file(path)
    # Add Category and Type
    bdtopo.insert(0, 'Category', path.split('/')[-2])
    bdtopo.insert(1, 'Type', path.split('/')[-1][:-4])
    print (f"Ajouté : {path.split('/')[-1]}")
    return bdtopo

# Chargement de tous les fichiers dans le dataframe bdtopo
paths_shp = glob.glob(PATHS_BDTOPO)
bdtopo = load_shp(paths_shp[0])
for path in paths_shp[1:]:
    bdtopo = pd.concat([bdtopo, load_shp(path)])
bdtopo = bdtopo.reset_index(drop=True)
print (f'BDTOPO chargé, taille : {len(bdtopo)}')

Ajouté : CONSTRUCTION_SURFACIQUE.shp
Ajouté : CONSTRUCTION_LINEAIRE.shp
Ajouté : CONSTRUCTION_PONCTUELLE.shp
Ajouté : TERRAIN_DE_SPORT.shp
Ajouté : TOPONYMIE_BATI.shp
Ajouté : LIGNE_OROGRAPHIQUE.shp
Ajouté : BATIMENT.shp
Ajouté : PYLONE.shp
Ajouté : CIMETIERE.shp
Ajouté : RESERVOIR.shp
Ajouté : POINT_DU_RESEAU.shp
Ajouté : AERODROME.shp
Ajouté : ITINERAIRE_AUTRE.shp
Ajouté : POINT_DE_REPERE.shp
Ajouté : EQUIPEMENT_DE_TRANSPORT.shp
Ajouté : TRONCON_DE_VOIE_FERREE.shp
Ajouté : TRONCON_DE_ROUTE.shp
Ajouté : TOPONYMIE_TRANSPORT.shp
Ajouté : NON_COMMUNICATION.shp
Ajouté : ROUTE_NUMEROTEE_OU_NOMMEE.shp
Ajouté : VOIE_NOMMEE.shp
Ajouté : SECTION_DE_POINTS_DE_REPERE.shp
Ajouté : PISTE_D_AERODROME.shp
Ajouté : VOIE_FERREE_NOMMEE.shp
Ajouté : ARRONDISSEMENT_MUNICIPAL.shp
Ajouté : COMMUNE.shp
Ajouté : REGION.shp
Ajouté : COLLECTIVITE_TERRITORIALE.shp
Ajouté : COMMUNE_ASSOCIEE_OU_DELEGUEE.shp
Ajouté : ARRONDISSEMENT.shp
Ajouté : EPCI.shp
Ajouté : DEPARTEMENT.shp
Ajouté : TOPONYMIE_ZONES_REGLEMENTEE

In [6]:
# Transformations

print (f'Dataframe bdtopo initial, taille : {len(bdtopo)}')

# Ajout d'un champ GeoType indiquant le type d'objet géométrique
bdtopo['GeoType'] = bdtopo['geometry'].geom_type

# Nettoyage pour ne conserver que les Polygon et MultiPolygon
bdtopo = bdtopo[bdtopo['GeoType'].isin(['Polygon', 'MultiPolygon'])]
print (f'Conservation des Polygon et MultiPolygon uniquement, taille : {len(bdtopo)}')

# Nettoyage pour ne conserver que les BATIMENT et RESERVOIR
bdtopo = bdtopo[bdtopo['Type'].isin(['BATIMENT', 'RESERVOIR'])].reset_index()
print (f'Conservation des BATIMENT ET RESERVOIR, taille : {len(bdtopo)}')

Dataframe bdtopo initial, taille : 996522
Conservation des Polygon et MultiPolygon uniquement, taille : 743190
Conservation des BATIMENT ET RESERVOIR, taille : 694302


In [7]:
# transformMultiPolygons() remplace les multi-polygones du dataframe par des polygones
# (en ajoutant des lignes) au dataframe

def transformMultiPolygonsIntoPolygons(df):
    mplines = df[df['GeoType']=='MultiPolygon']
    geoindex = mplines.columns.get_loc('geometry')
    dfpoly = gpd.GeoDataFrame(columns=['ID', 'geometry'])
    all_IDs, all_polygons = [], []
    
    for index, row in mplines.iterrows():
        polygons = list(row.iloc[geoindex].geoms)
        all_IDs = all_IDs + [row['ID']]*len(polygons)
        all_polygons = all_polygons + polygons

    all_polys_df = gpd.GeoDataFrame({'ID' : all_IDs, 'geometry' : all_polygons})
    df = df.merge(right=all_polys_df, on='ID', how='left')
    df = df.rename({'geometry_x' : 'geometry'}, axis=1)
    df.loc[df['GeoType']=='MultiPolygon', 'geometry'] = df.loc[df['GeoType']=='MultiPolygon', 'geometry_y']
    df = df.drop(columns=['geometry_y'])
    df = gpd.GeoDataFrame(df)
    return df

print (f'Nb lignes dataframe avant transformation des multi-polygones en polygones : {len(bdtopo)}')
bdtopo = transformMultiPolygonsIntoPolygons(bdtopo)
print (f'Nb lignes dataframe après transformation des multi-polygones en polygones : {len(bdtopo)}')

Nb lignes dataframe avant transformation des multi-polygones en polygones : 694302
Nb lignes dataframe après transformation des multi-polygones en polygones : 694950


In [6]:
# Sauvegarde du dataframe au format pickle .pkl

path = PATHS_BDTOPO_PICKLE + 'dft.pkl'
bdtopo.to_pickle(path)
print (f'DataFrame sauvegardé sous : {path}')

DataFrame sauvegardé sous : /Users/u600141/OneDrive - La Francaise des Jeux/Data/constructions/images/BDTOPO/pickle/bdtopo.pkl
